# eCREST_notebook

Basic functions for reconstructing cells.

# Setup

Do the following two setup steps regardless of how you will be using this script. 

## 1. Imports

Run the following code cell to import the necessary packages and modules. 

In [1]:
# %matplotlib ipympl

In [1]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique,nan
import pandas as pd
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import neuroglancer
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('/Users/kperks/Documents/ell-connectome/efish_em/efish_em')

In [3]:
# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest
import AnalysisCode as efish

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to do things like:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- add or remove annotation layers (see "Annotation Layers")
- check for overlap with other .json files in a directory folder (see "check for overlap")
- label cell structures
- add base_segments from a list (see "add segments")
- import annotations from another file (see "Annotation Import")
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


## 2. Settings definitions

Whether you are converting from neuroglancer or creating a new reconstruction, the settings_dict parameters is needed to create CREST json files with correct formatting. 
- 'save_dir' : the directory where JSON files are saved 
- 'cred' and 'db_path' : specify the path to the agglomeration database file on your local computer. 

In [4]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = efish.import_settings(path_to_settings_json)

In [5]:
settings_dict

{'save_dir': '/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/CREST_reconstructions/mg-network',
 'max_num_base_added': 1000,
 'cell_structures': ['unknown',
  'axon',
  'basal dendrite',
  'apical dendrite',
  'dendrite',
  'multiple'],
 'annotation_points': ['exit volume',
  'natural end',
  'uncertain',
  'pre-synaptic',
  'post-synaptic'],
 'db_path': '/Users/kperks/Documents/ell-connectome/eCREST-local-files/Mariela_bigquery_exports_agglo_v230111c_16_crest_proofreading_database.db'}

# Proofread using (e)CREST

The ```ecrest``` class defined in eCREST_cli.py can be used to proofread base_segment reconstructions enhanced by the agglomeration database.

An instance of this class can be initialized with either:
- ecrest(segment_id): a "main_base_id" in *int* format
- ecrest(filepath): an existing CREST .json file
- ecrest(segment_id, segment_list): the main_base_id from the neuroglancer file you are converting and a list of base_segments.

The ```launch_viewer``` flag default is "False" so that you can interact with the contents of a reconstruction without actually opening it visually in a neuroglancer tab. **NOTE**: Some ecrest functions require that the ecrest instance is created with ```launch_viewer==True```.

## Get base_segments dictionaries 

### Read from file (created with [this Colab notebook](https://colab.research.google.com/drive/19N8taRKeTt_Bgx_yF5AD5ntkU7zy7unc?usp=sharing)) 

In [ ]:
# Read list of dictionaries from a JSON file
with open(Path(settings_dict['save_dir']) /'Rachel/dictionary_jsons/dictionaries.json', 'r') as json_file: #assumes your save_dir is /mg_network/Rachel/ on your working computer
    dict_all = json.load(json_file)
  
dict_list = dict_all.keys()

# Convert lists to sets within each dictionary in data
for key, dictionary in dict_all.items():
    for sub_key, value in dictionary.items():
        if isinstance(value, list):
            dictionary[sub_key] = set(value)

# Dynamically create dictionaries based on the JSON keys
for key, value in dict_all.items():
    globals()[key] = value

print(dict_list)

### Create locally

In [435]:
# create base segment dictionaries to check for duplicates
# use manually-defined function in setup for "get_base_segments_dict, which can check for files with same id)"
base_segments_net = efish.get_base_segments_dict(Path(settings_dict['save_dir'])) 
base_segments_todo1 = efish.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo/mg1_pre') 
# base_segments_todo1 = get_base_segments_dict(Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/CREST_reconstructions/mg-network/todo/mg2_pre'))#
# base_segments_todo2 = get_base_segments_dict(Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/Ela/PE_post/Unlabled_completed_tracings')) 

## NEW reconstruction from segment ID

If you wanted to start reconstructing a new cell from a main base segment, you would use the following code block to launch.

To change the save location you can specify the ```directory_path``` flag in the ```save_cell_graph()``` module

In [397]:
len(get_cell_filepaths(Path(settings_dict['save_dir']) / 'todo/mg1_pre'))# mg2_386426114_pre')) # 'Rachel')) #

20

In [502]:
segment_id = '393387128'#'366760979'#     283375247
crest = ecrest(settings_dict,segment_id = segment_id, launch_viewer=True)

crest.change_key_binding({"alt+mousedown0" : "add-or-remove-seg"})

crest.add_endpoint_annotation_layers(['soma'],link=True) # spine_inputs
print(crest.get_ctype('manual'))

### check for duplicates in working directories

Creating base segment graph for cell 393387128 Cell Reconstruction
all base locations for 9 obtained from SQL database
graph created among all_base_segs
1 clusters of connected components. Connecting these clusters with nearest base segments.
weak clusters connected
segments without a location connected
1 clusters in graph (note should/would be only 1 if loaded base ID from agglomo fresh)
Created a CREST instance for NEW Reconstruction of 393387128. No file saved yet -- save manually.
updating viewer status message: Current Base Segment Counts: unknown: 9, axon: 0, basal dendrite: 0, apical dendrite: 0, dendrite: 0, multiple: 0
[]


In [499]:
# print(f'this file is: {cell_filepaths[cell_id].name}')
print('')
# actually check for duplicates of current reconstruction with base segments dictionary
# to save time, this line can be run alone after initializing base_segments dictionary above
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo1); display(df)
# print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo2); display(df)


overlap in main network:


,self,dups,overlap-percent,number_seg_lap


overlap in todo folder:


,self,dups,overlap-percent,number_seg_lap


### define cell type and save

In [500]:
cell_type = 'grc' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [501]:
crest.save_cell_graph() #directory_path = Path(settings_dict['save_dir']).parent.parent / 'VAST/VAST_consolidate_reconstructed/json_states/pf_post')

Saved cell 310963758 reconstruction locally at 2025-05-18 17.25.55


In [58]:
[p*r/v for p,r,v in zip((2151, 1552, 759),(128,128,120), (16,16,30))]

[17208.0, 12416.0, 3036.0]

In [16]:
Path(settings_dict['save_dir']).parent.parent / 'VAST/json_states/pf_post'

PosixPath('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/json_states/pf_post')


## EDIT reconstruction from file

If you wanted to edit a reconstruction from an existing file, you would use the following code block to launch.

Specify the cell_id and the path to the directory that cell is in. 

> NOTE: You can also directly copy paste the full filepath to the cell you want to open and pass it to the ```filepath``` flag.  
In that case, the only code you need is crest = ecrest(settings_dict,filepath= [*paste filepath here*], launch_viewer=True)

To change the save location you can specify the ```directory_path``` flag in the ```save_cell_graph()``` module

> To overwrite a file (not recommended), specify ```(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)```

In [163]:
Path(settings_dict['save_dir']).parent

PosixPath('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/CREST_reconstructions')

In [6]:
# directory_path = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/Ela/PE_post/Unlabled_completed_tracings')
directory_path = Path(settings_dict['save_dir']) #Path(settings_dict['save_dir']).parent / 'Spine_Density_Annotation/random_spines'#.parent / 'skin_RF'#/'Rachel' # specify the directory path

cell_filepaths = efish.get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory

In [ ]:
[126370492, 130842242, 131275458, 132266158, 132342130, 133255080,
       133427854, 134201244, 134787795, 135578944, 214476153, 215757263,
       215865536, 215913221, 216050665, 216607169, 217784485, 218528971,
       219485274, 220044275, 220817161, 221561350, 300674902, 304064921,
       304389729, 305130656, 305208703, 306103166, 306181538, 307375579,
       307377189, 308952373, 384167539, 387757100, 388575999, 389643522,
       390217833, 390848964, 391005614, 391066572, 391315890, 391748007,
       391856208, 392135367, 397178561,  46032263,  46669842, 472346357,
       472376328, 472501560,  47412870, 475455796, 475534424, 475672294,
       475859281,  47646140, 476739767, 478503314, 479587480, 480591852,
        48832843, 560124699, 560403307, 560448565, 563945764, 647607372,
       648615872, 648659573, 650548693, 650689301]

In [8]:
cell_id = '386612701' # specify the cell id

crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=True) #
print(cell_filepaths[cell_id])

crest.change_key_binding({"alt+mousedown0" : "add-or-remove-seg"})

updating viewer status message: Current Base Segment Counts: unknown: 2335, axon: 164, basal dendrite: 156, apical dendrite: 1348, dendrite: 0, multiple: 0
/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/CREST_reconstructions/mg-network/cell_graph_386612701__2025-05-12 16.34.45.json


Map({"dblclick0": "add-or-remove-seg", "alt+mousedown2": "mark-branch-in-colour", "shift+mousedown2": "change-anchor-seg", "alt+mousedown0": "add-or-remove-seg"})

In [494]:
t = crest.get_ctype('manual'); print(f'manual type: {t}')
t = crest.get_ctype('auto'); print(f'auto type: {t}')

manual type: uk
auto type: []


In [495]:
# check for duplicates in working directories

print(f'this file is: {cell_filepaths[cell_id].name}')
print('')
# actually check for duplicates of current reconstruction with base segments dictionary
# to save time, this line can be run alone after initializing base_segments dictionary above
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo1); display(df)
# print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo2); display(df)

this file is: cell_graph_400116643__2024-07-06 09.48.39.json

overlap in main network:


,self,dups,overlap-percent,number_seg_lap
3083,400116643,cell_graph_400116643__2024-07-06 09.48.39.json,1.0,60.0


overlap in todo folder:


,self,dups,overlap-percent,number_seg_lap


In [455]:
# crest.add_endpoint_annotation_layers(['soma'],link=True) # spine_inputs
print(crest.get_ctype('manual'))

mli


In [61]:
cell_type = 'grc' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [508]:
# SAVE YOUR WORK!

crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

Saved cell 391005614 reconstruction locally at 2025-05-19 16.08.44


In [23]:
crest.max_num_base_added = 3000

In [9]:
# directory_path = Path(settings_dict['save_dir']).parent.parent / 'Hannah/todo'

PosixPath('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/Hannah/todo')

In [67]:
# do not use unless certain - this overwrites the original file rather than making a new one
crest.save_cell_graph(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)

Saved cell 463095118 reconstruction locally at 2024-12-03 16.28.31


## Define Cell type

The cell type strings to use are:  
aff, grc-d, grc-s, sgx1, sgx2, sg1, sg2, mg1, mg2, lf, lg, uk, fov

You can also check the current cell type assigned by using ```crest.get_ctype("manual")```

In [ ]:
cell_type = 'grc' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

## Change Function Keybindings

speficy any keybindings that you want in ```keybindings_dict``` and pass that to the class module ```change_key_binding```. The keybinding mapping will be returned. 

In [ ]:
keybindings_dict = {
    "alt+mousedown0" : "add-or-remove-seg"
}

crest.change_key_binding(keybindings_dict)

## Check for overlap with other .json files in a directory

Use the ecrest module ```get_base_segments_dict``` to get a dictionary of {'cell_id' : 'base_segments'} for all .json files in a folder.  
Then use the ecrest module ```check_duplicates``` to get a dataframe of any instances of overlap between that cell and the .json files from the folder.

*Tips*:
- You do ***not*** need to run ```get_base_segments_dict``` every time. For each new crest instance you create, you can just skip to the ```check_duplicates``` step.
- You can create multiple dictionaries to check against... you can create a separate line of code for each. 

In [ ]:
base_segments_net = get_base_segments_dict(Path(settings_dict['save_dir']))
base_segments_todo = get_base_segments_dict(Path(settings_dict['save_dir']) / 'kp/392042360_grc-s_pre')

In [ ]:
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo); display(df)

## Add/Remove Annotation layers

Because of how CREST saves the .json state, annotation layers need to be added/removed programatically rather than via the neuroglancer viewer directly.

Comment/uncomment the following two module implementations as needed.

In [ ]:
crest.add_endpoint_annotation_layers(['skeleton'])

# crest.del_endpoint_annotation_layers(['soma'])

## spine density annotation functions

If you want to be able to move annotations, don't change alt+mouse0 keybinding

In [ ]:
crest.add_endpoint_annotation_layers(['volume'],link=False) # spine_inputs

In [ ]:
l = 5000 # distance along dendrite in nm

crest.add_endpoint_annotation('spineD loc',to_vox=False, center = [17497, 6482, 2001], radii = [l/16,l/16,l/30]) # 'center' is in middle of dendrite in voxels

## SAVE YOUR WORK!

In [ ]:
crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

# Other

## get random location within bbox

In [87]:
x_bounds = (10500, 22500)
y_bounds = (5000, 10000)#14744)
z_bounds = (1000, 2500)

random_point = random_location_3d(x_bounds, y_bounds, z_bounds)
print(random_point[0],random_point[1],random_point[2])

17901 7246 2252


## search for synapse by segment annotated

In [ ]:
directory_path = Path(settings_dict['save_dir']) #/ 'kp/pfs'#/ 'volume-subsample-all/in_progress'  ## specify the directory path
cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory
# cell_filepaths = cell_filepaths_mainnet

In [ ]:
         
syn_type = 'post-synaptic' #'spine_inputs' #
segs_to_find = ['564132646','565215897']
cells_todo = list(cell_filepaths.keys())

In [ ]:
vx_sizes = [16, 16, 30]

for c_id in cells_todo:
    crest = ecrest(settings_dict,filepath= cell_filepaths[c_id], launch_viewer=False)
    for syn_ in crest.cell_data['end_points'][syn_type]:
        try:
            if syn_[3] in segs_to_find:
                print(f'in cell {c_id}, segment {syn_[3]} is at synapse location {array([int(syn_[i]/vx_sizes[i]) for i in range(3)])}')

        except IndexError as msg:
            print(msg, f'for cell {c_id} synapse at {array([int(syn_[i]/vx_sizes[i]) for i in range(3)])} has no segment id')

## cell types for all files in directory

In [ ]:
nodefiles = get_cell_filepaths(Path(settings_dict['save_dir'])) 

In [ ]:
cell_type = {}
not_typed = []
for x,f in nodefiles.items():
    cell = ecrest(settings_dict,filepath = f,launch_viewer=False)
    cell_type[x] = cell.get_ctype('manual') 
    if (cell.get_ctype('manual') == []) | (cell.get_ctype('manual') == ''):
        cell_type[x]=''
        not_typed.append(x)# print(f'cell {x} is not cell-typed in json')
        
print('the following cells are not typed in the main network')
print(not_typed)        
        

## if get assertion error and won't save

In [ ]:
seglist = crest.cell_data['base_segments']['unknown']

anchor = crest.cell_data['metadata']['main_seg']['base']

crest2 = ecrest(settings_dict,segment_id = anchor, segment_list = seglist, launch_viewer=True)

# SAVE YOUR WORK!

crest2.cell_data['end_points']=crest.cell_data['end_points']


crest2.load_annotation_layer_points()

crest2.save_cell_graph()

## volume of annotation points

In [ ]:
neuroglancer_path = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-kperky@gmail.com/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer/pf_synapses/pf_282137001_213811_154407_44715.json')
# neuroglancer_path = '/Users/kperks/Library/CloudStorage/GoogleDrive-kperky@gmail.com/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer/synaptic_labeling/finished'
# filename = '387850816_type2spectrum_nbs.json'
# neuroglancer_path = Path('/Users/kperks/Downloads/grc_483610898.json') # Path(neuroglancer_path) / filename # 

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)

In [ ]:
from scipy.spatial import ConvexHull
# hull_of_df1 = ConvexHull(df1)

In [ ]:
nl_ = 'terminals'
neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)
df_points_terminal = pd.DataFrame([[v['point'][0]*16/1000,v['point'][1]*16/1000,v['point'][2]*30/1000] for v in neuroglancer_layer['annotations']] ,columns=['x','y','z'])
hull_terminal = ConvexHull(df_points_terminal)

nl_ = 'spines'
neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)
df_points_spine = pd.DataFrame([[v['point'][0]*16/1000,v['point'][1]*16/1000,v['point'][2]*30/1000] for v in neuroglancer_layer['annotations']] ,columns=['x','y','z'])
hull_spine = ConvexHull(df_points_spine)

In [ ]:
%matplotlib ipympl

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_points_terminal['x'], df_points_terminal['y'], df_points_terminal['z'],color='black')
for i in hull_terminal.simplices:
    plt.plot(hull_terminal.points[i,0], hull_terminal.points[i,1], hull_terminal.points[i,2], 'r-')

ax.scatter(df_points_spine['x'], df_points_spine['y'], df_points_spine['z'],color='black')
for i in hull_spine.simplices:
    plt.plot(hull_spine.points[i,0], hull_spine.points[i,1], hull_spine.points[i,2], 'g-')



In [ ]:
plt.close('all')

In [ ]:
hull_spine.volume

In [ ]:
hull_terminal.volume

## eCREST from json segments

### neuroglancer file

In [17]:
# neuroglancer_path = Path(settings_dict['save_dir']) / 'Nate_neuroglancer/synaptic_labeling/finished'
neuroglancer_path = '/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/Ela/PE_post/Unlabled_completed_tracings/'
filename = 'cell_graph_645258321__2025-02-10 09.42.17.json' #'386178187_mg_nbs.json'
neuroglancer_path = Path(neuroglancer_path) / filename # Path('/Users/kperks/Downloads/grc_483610898.json') # 

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)

segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
base_segment_list_ng = set(segmentation_layer['segments'])

base_segment_list_ng = list(set([bs for bs in base_segment_list_ng if '!' not in bs]))# -set(['283547027']))

In [30]:
cell_id = '213605530' # specify the cell id
crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=False) #


In [32]:
all_segs = set([a for b in crest.cell_data['base_segments'].values() for a in b])

In [37]:
len(all_segs)

8628

In [33]:
cell_id = '213621408' # specify the cell id
crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=False) #

In [36]:
all_segs.update(set([a for b in crest.cell_data['base_segments'].values() for a in b]))

In [38]:
segment_id = 213605530
crest = ecrest(settings_dict, segment_id = segment_id, segment_list = list(all_segs), launch_viewer=True)

Creating base segment graph for cell 213605530 Cell Reconstruction
all base locations for 8628 obtained from SQL database
graph created among all_base_segs
876 clusters of connected components. Connecting these clusters with nearest base segments.


KeyboardInterrupt: 

In [594]:
'''
Currently only works for annotation layers that you know contain only point annotations!!!
'''
neuroglancer_layer_name = ['uncertain','natural end','pre-synaptic','post-synaptic','soma','exit volume']#['Base Segment Merger']#['skel','post-synaptic','branch_traced']#, 'natural end','exit volume','uncertain']# set([item['name'] for item in neuroglancer_data['layers'] if item['type']=='annotation'])-set(['Base Segment Merger'])
crest_layer_name = neuroglancer_layer_name #['post-synaptic','pre-synaptic']#,

for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):
    neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)
    
    if neuroglancer_layer != None:
        # if cl_ in crest.point_types:

        if cl_ == 'Base Segment Merger':
            this_type_points = crest.cell_data['base_seg_merge_points']
        else:
            this_type_points = crest.cell_data['end_points'][cl_]
            
        for v in neuroglancer_layer['annotations']: 

            co_ords = [float(x) for x in v['point']]
            co_ords_and_id = ([co_ords[x]*crest.vx_sizes['em'][x] for x in range(3)])

            if 'segments' in v.keys(): #v['segments'] != None:
                if len(v['segments'][0]) > 0:
                    co_ords_and_id.append(str(v['segments'][0][0]))

            this_type_points.append((co_ords_and_id,'annotatePoint'))

        if cl_ == 'Base Segment Merger':
            this_type_points = crest.cell_data['base_seg_merge_points'] + this_type_points
            this_type_points = [(tuple(inner_list), label) for inner_list, label in this_type_points]
            # Use a set to track unique elements based on the converted tuples
            unique_end_points_set = set(this_type_points)
            # Convert inner tuples back to lists for the final result
            this_type_points = [(list(inner_tuple), label) for inner_tuple, label in unique_end_points_set]
            
            crest.cell_data['base_seg_merge_points'] = this_type_points

        else: 
            # remove duplicates with annotations already existing
            # Convert inner lists to tuples
            this_type_points = crest.cell_data['end_points'][cl_] + this_type_points
            this_type_points = [(tuple(inner_list), label) for inner_list, label in this_type_points]
            # Use a set to track unique elements based on the converted tuples
            unique_end_points_set = set(this_type_points)
            # Convert inner tuples back to lists for the final result
            this_type_points = [(list(inner_tuple), label) for inner_tuple, label in unique_end_points_set]
    
    
            crest.cell_data['end_points'][cl_] =  this_type_points
        # else: 
        #     msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
        #     print(msg)

    else:
        msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
        print(msg)
        
# crest.load_annotation_layer_points()
crest.set_base_seg_merger_layer()

### initiate cell if not already a crest file

In [18]:
segment_id = filename.split('_')[2].split('.')[0]
crest = ecrest(settings_dict, segment_id = segment_id, segment_list = base_segment_list_ng, launch_viewer=True)

Creating base segment graph for cell 645258321 Cell Reconstruction
all base locations for 440 obtained from SQL database
graph created among all_base_segs
31 clusters of connected components. Connecting these clusters with nearest base segments.
weak clusters connected
segments without a location connected
1 clusters in graph (note should/would be only 1 if loaded base ID from agglomo fresh)
Created a CREST instance for NEW Reconstruction of 645258321. No file saved yet -- save manually.
updating viewer status message: Current Base Segment Counts: unknown: 440, axon: 0, basal dendrite: 0, apical dendrite: 0, dendrite: 0, multiple: 0


In [591]:
crest.add_endpoint_annotation_layers(['skel','branch_traced'])

# crest.del_endpoint_annotation_layers(['Base Segment Merger'])

In [592]:
crest.load_annotation_layer_points()

### Add points from annotation layers

In [ ]:
set([item['name'] for item in neuroglancer_data['layers'] if item['type']=='annotation'])-set(['Base Segment Merger'])

In [566]:
def add_merge_annotation_layers(crest,layer_names=['Base Segment Merger'], color='#ffa500', link=True):
    
    if crest.launch_viewer == True:
        if not crest.save_point_types_successfully(): 
            return

    crest.point_types = list(set(crest.point_types + list(crest.cell_data['end_points'].keys()) + layer_names))
    # self.point_types = [x for x in self.point_types if not ('base' in x.lower() and 'merge' in x.lower())]

    for t in layer_names:
        if t not in crest.cell_data['end_points'].keys():
            crest.cell_data['end_points'][t] = []

            with crest.viewer.txn(overwrite=True) as s:
                for point_type in layer_names:
                    s.layers[point_type] = neuroglancer.AnnotationLayer()
                    s.layers[point_type].tool = "annotatePoint"
                    s.layers[point_type].tab = 'Annotations'
                    s.layers[point_type].annotationColor = '#ffffff'

                    if link==True:
                        s.layers[point_type].linkedSegmentationLayer = {"segments": 'base_segs'} # set it up linked to base_segs
                        s.layers[point_type].filterBySegmentation = ["segments"]
                    if color != None:
                        s.layers[point_type].annotationColor = color


In [594]:
'''
Currently only works for annotation layers that you know contain only point annotations!!!
'''
neuroglancer_layer_name = ['Base Segment Merger']#['skel','post-synaptic','branch_traced']#, 'natural end','exit volume','uncertain']# set([item['name'] for item in neuroglancer_data['layers'] if item['type']=='annotation'])-set(['Base Segment Merger'])
crest_layer_name = neuroglancer_layer_name #['post-synaptic','pre-synaptic']#,

for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):
    neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)
    
    if neuroglancer_layer != None:
        # if cl_ in crest.point_types:

        if cl_ == 'Base Segment Merger':
            this_type_points = crest.cell_data['base_seg_merge_points']
        else:
            this_type_points = crest.cell_data['end_points'][cl_]
            
        for v in neuroglancer_layer['annotations']: 

            co_ords = [float(x) for x in v['point']]
            co_ords_and_id = ([co_ords[x]*crest.vx_sizes['em'][x] for x in range(3)])

            if 'segments' in v.keys(): #v['segments'] != None:
                if len(v['segments'][0]) > 0:
                    co_ords_and_id.append(str(v['segments'][0][0]))

            this_type_points.append((co_ords_and_id,'annotatePoint'))

        if cl_ == 'Base Segment Merger':
            this_type_points = crest.cell_data['base_seg_merge_points'] + this_type_points
            this_type_points = [(tuple(inner_list), label) for inner_list, label in this_type_points]
            # Use a set to track unique elements based on the converted tuples
            unique_end_points_set = set(this_type_points)
            # Convert inner tuples back to lists for the final result
            this_type_points = [(list(inner_tuple), label) for inner_tuple, label in unique_end_points_set]
            
            crest.cell_data['base_seg_merge_points'] = this_type_points

        else: 
            # remove duplicates with annotations already existing
            # Convert inner lists to tuples
            this_type_points = crest.cell_data['end_points'][cl_] + this_type_points
            this_type_points = [(tuple(inner_list), label) for inner_list, label in this_type_points]
            # Use a set to track unique elements based on the converted tuples
            unique_end_points_set = set(this_type_points)
            # Convert inner tuples back to lists for the final result
            this_type_points = [(list(inner_tuple), label) for inner_tuple, label in unique_end_points_set]
    
    
            crest.cell_data['end_points'][cl_] =  this_type_points
        # else: 
        #     msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
        #     print(msg)

    else:
        msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
        print(msg)
        
# crest.load_annotation_layer_points()
crest.set_base_seg_merger_layer()

In [132]:
crest.point_types = ['exit volume',
 'pre-synaptic',
 'uncertain',
 'natural end',
 'post-synaptic',
 'skel',
 'Base Segment Merger']

In [595]:
crest.cell_data['base_seg_merge_points']

[([350135.875, 182104.890625, 67545.0, '458759898'], 'annotatePoint'),
 ([353610.375, 165572.359375, 83415.00732421875, '542361707'],
  'annotatePoint'),
 ([354393.0, 177353.0625, 67905.0, '458775517'], 'annotatePoint'),
 ([349775.3125, 182519.140625, 67755.0, '458759898'], 'annotatePoint'),
 ([350202.3125, 181439.671875, 67635.0, '458775406'], 'annotatePoint')]

In [ ]:
# crest.cell_data['end_points']= crest_ann.cell_data['end_points']

In [ ]:
# crest.load_annotation_layer_points()

### define type and save

In [596]:
cell_type = 'pf' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [110]:
# SAVE YOUR WORK!

crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

Saved cell 277947665 reconstruction locally at 2025-02-14 10.48.18


In [ ]:
# crest.add_endpoint_annotation_layers(['soma'])

In [ ]:
crest.cell_data['end_points']['uncertain'] = [r for r in crest.cell_data['end_points']['uncertain'] if r[1] < (11100*16)]

In [ ]:
crest.load_annotation_layer_points()

In [ ]:
base_segments_net = crest.get_base_segments_dict(Path(settings_dict['save_dir']))
base_segments_todo = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_presynaptic/lf_393325331')

In [ ]:
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo); display(df)

In [ ]:
cell_type = 'sgx1' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [ ]:
# SAVE YOUR WORK!

crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

In [ ]:
# cell_filepaths[cell_id].name

In [ ]:
crest.get_ctype('manual')

### compare base segments with ecrest file

In [78]:
directory_path = Path(settings_dict['save_dir']) #/ 'todo_presynaptic/mg1_299496636' #/ 'kp/pfs'#/ 'volume-subsample-all/in_progress'  ## specify the directory path
cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory
# cell_filepaths = cell_filepaths_mainnet

cell_id = '290280494' #filename.split('_')[0]#'483610898'#
# crest = ecrest(settings_dict, segment_id = segment_id, segment_list = base_segment_list_ng, launch_viewer=True)

crest = ecrest(settings_dict, filepath= cell_filepaths[cell_id], launch_viewer=True)

print(set(base_segment_list_ng) - set([a for _,b in crest.cell_data['base_segments'].items() for a in b]))

updating viewer status message: Current Base Segment Counts: unknown: 430, axon: 333, basal dendrite: 10, apical dendrite: 21, dendrite: 0, multiple: 0
{'291425375', '290281283', '291425871', '376149007', '291425485', '290280446', '377293797', '377294049', '377293815', '376149008', '376149385', '291425414', '377293734', '376149046', '290281185', '376149108', '290281106', '377293706', '377294148', '377293671', '291425028', '291425633', '377293643', '376149099', '290281122', '377293751', '376149076', '376149077', '290281192', '290280998', '377293722', '377294211', '377293753', '291425281', '377294151', '377294182', '291425677', '376149344', '377293782', '291425856', '376149097', '376149055', '377294291', '377293673', '377294193', '291425859', '290280607', '377278585', '291425648', '290281068', '377294269', '291425645', '377293739', '377293768', '376149009', '376149346', '377294083', '290281182', '377293713', '290280609', '376149331', '291425820', '376149105', '291425872', '291425632', '3

### add segments if needed

In [79]:
base_ids_added = set()
for base_seg in set(base_segment_list_ng) - set([a for _,b in crest.cell_data['base_segments'].items() for a in b]):
    
    if (base_ids_added&set(base_seg)==set()) & (base_seg != crest.cell_data['metadata']['main_seg']['base']): 
        
        displayed_segs = crest.assert_segs_in_sync(return_segs=True)
        if base_seg in displayed_segs:
            # print(f'{base_seg} already in cell, continueing')
            continue

        # print(i,base_seg)
        agglo_seg = crest.get_agglo_seg_of_base_seg(base_seg)

        constituent_base_ids = crest.get_base_segs_of_agglo_seg(agglo_seg)        
        current_segs = crest.assert_segs_in_sync(return_segs=True)

        num_base_segs_this_agglo_seg = len(constituent_base_ids)
        constituent_base_ids = [x for x in constituent_base_ids if x not in current_segs]
        constituent_base_ids = [x for x in constituent_base_ids if x not in crest.cell_data['removed_base_segs']]
        num_base_segs_not_already_included = len(constituent_base_ids)
        
        if len(constituent_base_ids) > crest.max_num_base_added:
            base_ids = [base_seg]
            # crest.large_agglo_segs.add(agglo_seg)
            print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {crest.max_num_base_added}')
            # print(f'{base_seg} part of an agglo seg {agglo_seg} that is too large to add, so just adding the one segment')
        else:
            base_ids = constituent_base_ids

        if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

            if not base_seg in base_ids:
                base_ids.append(base_seg)
        print(base_ids)
        crest.update_base_locations(base_ids)
        crest.pr_graph.add_vertices(base_ids)

        if len(base_ids) > 1:
            edges = crest.get_edges_from_agglo_seg(agglo_seg)
            edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
            crest.pr_graph.add_edges(edges)

        join_msg = crest.add_closest_edge_to_graph(base_ids, base_seg) 
        

        # Update lists of base segments and displayed segs:
        crest.cell_data['base_segments']['unknown'].update(set(base_ids))

        with crest.viewer.txn(overwrite=True) as s:

            for bs in base_ids:
                s.layers['base_segs'].segment_colors[int(bs)] = '#d2b48c'
                s.layers['base_segs'].segments.add(int(bs))
                
        base_ids_added.update(base_ids)


        crest.update_displayed_segs() 
        crest.assert_segs_in_sync()

['291425375']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['290281283']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['291425871']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['376149007']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['291425485']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['290280446']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['377293797']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['377294049']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['377293815']
1 clusters of connected components. Connecting these clusters with nearest base segments.
['376149008']
1 clusters of connected components. Connecting the

## remove annotations from a layer

In [878]:
crest.cell_data['end_points']['post-synaptic'] =  []

In [879]:
crest.load_annotation_layer_points()

## Molecular layer fit

In [ ]:
from scipy.optimize import curve_fit 
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D 

In [ ]:
# Define mathematical function for curve fitting 
def func(xy, a, b, c, d, e, f, g, h):  # #h):#
    x, y = xy 
    return a + b*x + c*y + d*x**2 + e*y**2 + f*x**3 + g*y**3 + h*x*y # + h*x*y #

In [ ]:
Path(settings_dict['save_dir']).parent.parent

In [ ]:
neuroglancer_path = Path(settings_dict['save_dir']).parent.parent / 'Krista/blender/soma_locations/layer-molecular_annotation.json'
voxel_sizes = [16,16,30]
nl_ = 'molecular'

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)
    
neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)
vertices = [[p['point'][i]*voxel_sizes[i] for i in range(3)] for p in neuroglancer_layer['annotations']]

x_pts = [p[0] for p in vertices]
y_pts = [p[1] for p in vertices]
z_pts = [p[2] for p in vertices]

In [ ]:
# Perform curve fitting 
popt, pcov = curve_fit(func, (x_pts, z_pts), y_pts) 
  
# Print optimized parameters 
print(popt) 

In [ ]:
# Create 3D plot of the data points and the fitted curve 
fig = plt.figure(figsize=(10,10)) 
ax = fig.add_subplot(111, projection='3d') 
ax.scatter(x_pts, z_pts, y_pts, color='blue') 
x_range = np.linspace(np.min(x_pts), np.max(x_pts), 100) 
z_range = np.linspace(np.min(z_pts), np.max(z_pts), 100) 
X, Z = np.meshgrid(x_range, z_range) 
Y = func((X, Z), *popt) 
ax.plot_surface(X, Z, Y, color='red', alpha=0.5) 
ax.set_xlabel('X') 
ax.set_ylabel('Z') 
ax.set_zlabel('Y') 
# plt.show()
ax.set_aspect('equal') 

## change cell type and re-save

### auto cell type defined different from manual

In [ ]:
128751464,215897023,

In [ ]:
for cell_id in [str(id) for id in [128751464]]:
    
    crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=False)
    t_man = crest.get_ctype('manual')

    if t_man =='sg1':
        crest.define_ctype('sg2',"auto")
    elif t_man =='sg2':
        crest.define_ctype('sg1',"auto")
    else: 
        print(f'{t_man}')

    crest.save_cell_graph(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)

## resave files to change owner

whichever account the google drive desktop application is currently logged in to, that will be the new owner (regardless of who you are signed into Google Chrome as... unless you are doing this in Google Colab... then I think it will be whoever you mounted the drive as)

In [ ]:
128751464,215897023,

In [ ]:
for cell_id in [str(id) for id in [128751464]]:
    
    crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=False)
    t_man = crest.get_ctype('manual')

    if t_man =='sg1':
        crest.define_ctype('sg2',"auto")
    elif t_man =='sg2':
        crest.define_ctype('sg1',"auto")
    else: 
        print(f'{t_man}')

    crest.save_cell_graph(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)

## random location generator

In [11]:
import random

def random_location_3d(x_range, y_range, z_range):
    """
    Returns a random location in 3D space within the given ranges.

    Args:
    x_range: tuple of (min_x, max_x) for the x dimension.
    y_range: tuple of (min_y, max_y) for the y dimension.
    z_range: tuple of (min_z, max_z) for the z dimension.

    Returns:
    A tuple (x, y, z) representing the random coordinates.
    """
    x = round(random.uniform(x_range[0], x_range[1]))
    y = round(random.uniform(y_range[0], y_range[1]))
    z = round(random.uniform(z_range[0], z_range[1]))
    
    return (x, y, z)

In [331]:
# Example usage
x_bounds = (5646, 26629)
y_bounds = (6000, 14744)
z_bounds = (200, 1300)

random_point = random_location_3d(x_bounds, y_bounds, z_bounds)
print(random_point)

(24443, 12484, 1299)
